# Modelling

In [26]:
from xgboost import XGBClassifier
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, precision_score, recall_score, f1_score

#machine learning
from xgboost import XGBClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from catboost import CatBoostClassifier
from sklearn.preprocessing import LabelEncoder

In [27]:
train_set = pd.read_csv("preselection_df.csv")
train_set = train_set.drop('Unnamed: 0', axis=1)

C:\Users\adamw\AppData\Local\Temp\ipykernel_5420\187807539.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  train_set = pd.read_csv("preselection_df.csv")


### Train/Test Split

In [28]:
train_set = train_set[train_set['status_group'].notna()]
test_set = train_set[train_set['status_group'].isna()]

###define X and y for training the model
X= train_set.drop(['id', 'status_group'], axis=1)
y = train_set['status_group']

encoder = LabelEncoder()
y = encoder.fit_transform(y)
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size = 0.2, random_state=42 , stratify=y)

### AdaBoost Ensemble

In [29]:
num_models = 5

# Step 4: Train the models
models = []
for i in range(num_models):
    model = XGBClassifier(max_depth =12, n_estimators= 200, colsample_bytree = 0.3, eta = 0.2, random_state = 42)  # Set desired hyperparameters
    model.fit(X_train, y_train)
    models.append(model)

# Step 5: Evaluate the models
for i, model in enumerate(models):
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Model {i+1} accuracy: {accuracy}")

# Step 6: Combine predictions from the ensemble
ensemble_predictions = []
for model in models:
    y_pred = model.predict(X_test)
    ensemble_predictions.append(y_pred)

# Perform an ensemble operation (e.g., averaging the predictions)
final_predictions = pd.DataFrame(ensemble_predictions).mean(axis=0)

# Print the final predictions
print(final_predictions)

Model 1 accuracy: 0.8137205387205387
Model 2 accuracy: 0.8137205387205387
Model 3 accuracy: 0.8137205387205387
Model 4 accuracy: 0.8137205387205387
Model 5 accuracy: 0.8137205387205387
0        0.0
1        2.0
2        0.0
3        2.0
4        2.0
        ... 
11875    2.0
11876    0.0
11877    0.0
11878    0.0
11879    2.0
Length: 11880, dtype: float64
